In [3]:
allOS = []

In [6]:
from cvzone.HandTrackingModule import HandDetector
import cv2
import boto3
def add_OS():
    ec2 = boto3.resource("ec2")
    instances = ec2.create_instances(
        ImageId="ami-0a2acf24c0d86e927",
        MinCount=1,
        MaxCount=1,
        InstanceType="t2.micro",
        SecurityGroupIds=["sg-061fc03509c386fb6"]
    )
    allOS.append(instances[0].id)
    print(len(allOS))
    
def deleteOS():
    ec2 = boto3.resource("ec2")
    if(len(allOS)>0):
        lastOS = allOS.pop()
        ec2.instances.filter(InstanceIds = [lastOS]).terminate()
        print(len(allOS))
    else:
        print("array empty")
    
cap = cv2.VideoCapture(0)
while(True):
    status, photo = cap.read()
#     crop = photo[100:400,100:400]
    cv2.imshow("my-image",photo)
    if (cv2.waitKey(1000) == 13):
        cap.release()
        cv2.destroyAllWindows()
        break;
    detector = HandDetector(maxHands=1)
    ishand = detector.findHands(photo,draw=False)
#     cv2.imshow("new photo",photo)
#     cv2.waitKey()
#     cv2.destroyAllWindows()
    # print(ishand[0])
    if(ishand):
        fingerlist = ishand[0]
        fingers = detector.fingersUp(fingerlist)
        print(fingers)
        print(ishand[0]["lmList"])
        x = ishand[0]["lmList"][4][0] - ishand[0]["lmList"][8][0]; # tip of thumb
        y = ishand[0]["lmList"][4][1] - ishand[0]["lmList"][8][1]; # tip of index finger
        if(abs(x)<=20 and abs(y)<=20): # if index and thumb are touching on tip, add an instance
            print("Touching")
            cv2.waitKey(3000)
            add_OS()
            cap.release()
            cv2.destroyAllWindows()
            break;
        elif(fingers == [0,1,1,0,0]): # if index and middle finger are shown, delete an instance
            print("2 fingers")
            cv2.waitKey(3000)
            deleteOS()
            cap.release()
            cv2.destroyAllWindows()
            break;
        else:
            print("not touching")
    else:
        print("no hands")

no hands
no hands
no hands
no hands
no hands
no hands
no hands
no hands
no hands
[0, 1, 1, 0, 0]
[[160, 396, 0], [212, 369, -33], [239, 318, -45], [205, 261, -55], [156, 235, -63], [242, 230, -14], [264, 160, -28], [278, 121, -41], [288, 84, -52], [196, 218, -12], [195, 148, -30], [191, 104, -44], [186, 65, -54], [158, 231, -14], [150, 172, -46], [156, 222, -50], [158, 261, -44], [122, 260, -18], [117, 223, -49], [127, 253, -51], [135, 283, -45]]
2 fingers
1
